In [1]:
import numpy as np
from simulation.generate_path import generate_path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from estimation.distance_sensor import distance_sensors
from estimation.non_iterative_estimator import non_iterative_estimator
from estimation.kalman_filter_from_points import kalman_filter_from_points
import random
import torch
import torch.nn as nn
import time

In [2]:
# create test data
target_initial_pos = np.array([0, 0, 5000])
target_speed_xy = 50
target_speed_z = 10
target_rot_speed = 3
time_res = 0.1

path1 = generate_path(0, target_speed_xy, target_speed_z, target_initial_pos, time_res)
path1.add_straight_interval(100)
path1.add_xy_turn_interval(90, -np.deg2rad(target_rot_speed))
path1.add_straight_interval(100)

# create noisy sensors
sensors = distance_sensors([[-5000,0,0],[ 400, -7400, 0],[ 800, 800, 0]], 15)
sensors.calculate_measurements(path1.path)

sample = 3
XTest = []
for i in np.arange(len(path1.path) - sample + 1):
    tmp = np.concatenate((sensors.sensor_locations, np.reshape(sensors.noisy_distances[i,:], (1,3))), 0)
    tmp = tmp.reshape(4,3,1)
    for j in np.arange(1,sample):
        matrix = np.concatenate((sensors.sensor_locations, np.reshape(sensors.noisy_distances[i+j,:], (1,3))), 0)
        matrix = matrix.reshape(4,3,1)
        tmp = np.concatenate((tmp, matrix),axis=2)
    if i > 0:
        tmp = tmp.reshape(4,3,sample,1)
        XTest = np.concatenate((XTest, tmp), 3)
    else:
        tmp = tmp.reshape(4,3,sample,1)
        XTest = tmp

YTest = path1.state_key[sample-1:]

print(np.shape(YTest))
print(np.shape(XTest))

(2896, 1)
(4, 3, 3, 2896)


In [ ]:
def make_outliers(point):
    point[0] += random.choice(np.random.randint(13, 20), np.random.randint(-20, -13))
    point[1] += random.choice(np.random.randint(13, 20), np.random.randint(-20, -13))
    point[2] += random.choice(np.random.randint(13, 20), np.random.randint(-20, -13))

In [7]:
# create train data
run_number = 50
for k in np.arange(run_number):
    target_initial_pos = np.random.randint(-7000, 7000, size=(1, 3))[0]
    sensors_pos = np.random.randint(-7000, 7000, size=(3, 3))[:,:]
    target_speed_xy = 50
    target_speed_z = 10
    target_rot_speed = 3
    time_res = 0.1

    path1 = generate_path(0, target_speed_xy, target_speed_z, target_initial_pos, time_res)
    path1.add_straight_interval(np.random.randint(0,100,size=1)[0])
    path1.add_xy_turn_interval(np.random.randint(0,100,size=1)[0], -random.choice([-1, 1])*np.deg2rad(target_rot_speed))
    outliers = np.random.randint(0, 100, size=len(path1.path))
    outliers = outliers > 90

    # create noisy sensors
    sensors = distance_sensors(sensors_pos, 15)
    sensors.calculate_measurements(path1.path)

    non_it_est = non_iterative_estimator(sensors, path1.path[0,:])
    estimated_path = non_it_est.estimate_path()

    sigma_a = 1
    sigma_v = 500
    kf = kalman_filter_from_points(time_res, sigma_a, sigma_v, non_diag_reduction_ratio=2)
    kf_path, P, X = kf.filter_path(estimated_path)

    sample = 3
    XTrain = []
    for i in np.arange(len(kf_path) - sample + 1):
        tmp = np.concatenate((sensors.sensor_locations, np.reshape(sensors.noisy_distances[i,:], (1,3))), 0)
        tmp = tmp.reshape(4,3,1)
        for j in np.arange(1,sample):
            matrix = np.concatenate((sensors.sensor_locations, np.reshape(sensors.noisy_distances[i+j,:], (1,3))), 0)
            matrix = matrix.reshape(4,3,1)
            tmp = np.concatenate((tmp, matrix),axis=2)
        if i > 0:
            tmp = tmp.reshape(4,3,sample,1)
            XTrain = np.concatenate((XTrain, tmp), 3)
        else:
            tmp = tmp.reshape(4,3,sample,1)
            XTrain = tmp

    YTrain = path1.state_key[sample-1:]

print(np.shape(YTrain))
print(np.shape(XTrain))



KeyboardInterrupt



In [264]:
# shuffle data
ind = np.arange(len(YTrain))
random.shuffle(ind)

XVal = torch.from_numpy(np.transpose(XTest[:,:,:,ind], (3, 2, 0, 1)))
YVal = torch.from_numpy(YTest[ind,:])

XTrain = torch.from_numpy(np.transpose(XTrain[:,:,:,ind], (3, 2, 0, 1)))
YTrain = torch.from_numpy(YTrain[ind,:])

In [265]:
# create network
class state_estimat(nn.Module):
    def __init__(self, d_in, num_classes):
        # initialzing the parent object (important!)
        super(state_estimat, self).__init__()
        # Create a pipeline - a sequence of layers
        self.pipe = torch.nn.Sequential(
            nn.Conv2d(d_in, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Linear(3, num_classes),
            nn.Softmax())

    def forward(self, x):
        return self.pipe(x)


In [269]:
# hyper-parameters:
num_epochs = 20
batch_size = 8
learning_rate = 0.001
learning_rate_drop_period = 3
input_shape = [3,4,3]

# Device configuration, as before
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# create model
model = state_estimat(d_in=3, num_classes=1).to(device)
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [270]:
# Train the model
total_step = len(YTrain)
start_time = time.time()
model = model.float()
for epoch in range(num_epochs):
    for i in range(int(total_step/batch_size)):
        # each i is a batch of 128 samples
        x, y = XTrain[i*batch_size:(i+1)*batch_size,:,:,:], YTrain[i*batch_size:(i+1)*batch_size,:]

        x = x.to(device)
        y = y.to(device)

        # Forward pass
        outputs = model(x.float())
        loss = criterion(outputs, y.float().T)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Time: {:.4f} secs'
                    .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), time.time() - start_time))

D:\Programs\Anaconda\envs\deep_learn\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
D:\Programs\Anaconda\envs\deep_learn\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([8, 64, 4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/20], Step [100/1877], Loss: 0.5000, Time: 0.6075 secs
Epoch [1/20], Step [200/1877], Loss: 0.6211, Time: 1.0815 secs
Epoch [2/20], Step [100/1877], Loss: 0.5000, Time: 1.6665 secs
Epoch [2/20], Step [200/1877], Loss: 0.6211, Time: 2.0481 secs
Epoch [3/20], Step [100/1877], Loss: 0.5000, Time: 2.6798 secs
Epoch [3/20], Step [200/1877], Loss: 0.6211, Time: 3.1241 secs
Epoch [4/20], Step [100/1877], Loss: 0.5000, Time: 3.7715 secs
Epoch [4/20], Step [200/1877], Loss: 0.6211, Time: 4.2313 secs
Epoch [5/20], Step [100/1877], Loss: 0.5000, Time: 4.9030 secs
Epoch [5/20], Step [200/1877], Loss: 0.6211, Time: 5.3872 secs
Epoch [6/20], Step [100/1877], Loss: 0.5000, Time: 6.0121 secs
Epoch [6/20], Step [200/1877], Loss: 0.6211, Time: 6.5120 secs
Epoch [7/20], Step [100/1877], Loss: 0.5000, Time: 7.2774 secs
Epoch [7/20], Step [200/1877], Loss: 0.6211, Time: 7.8242 secs
Epoch [8/20], Step [100/1877], Loss: 0.5000, Time: 8.4803 secs
Epoch [8/20], Step [200/1877], Loss: 0.6211, Time: 8.96